In [ ]:
# Set working directory to project root, identified by presence of the .Rproj file
import os
while not os.path.exists('workshop-pythonr.Rproj'):
    current_dir = os.getcwd()
    parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
    if current_dir == parent_dir:
        raise Exception('Can not find project root directory.')
    os.chdir('..')
print('Working directory set to:', os.getcwd())

# Imports

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import rpy2.robjects as robj
import rpy2.robjects.packages

from scripts import data

base = robj.packages.importr('base')
robj.r.source('scripts/analysis.R');

# Load data and prepare regression wrapper

In [ ]:
df = data.get_county_shape_df()
# keep only contiguous states to simplify mapping
df = df[~df['statefp'].isin(['02', '15', '60', '66', '69', '72', '78'])]
d = data.get_ui_df()
df = df.merge(d, how='inner', on='fips')

def show_model(yearmin, yearmax, rural_uics):
    """Estimate and print regression model, calling function defined in R script."""
    m = robj.r.estimate_model(yearmin, yearmax, rural_uics)
    s = base.summary(m)
    
    print('\nCoefficient estimates:')
    print(s.rx2('coefficients'))
    
    coef = dict(zip(m.rx2('coefficients').names, m.rx2('coefficients')))

    est_urban = coef['estabs_entry_rate'] + coef['estabs_exit_rate']
    est_rural = est_urban + coef['estabs_entry_rate:ruralTRUE'] + coef['estabs_exit_rate:ruralTRUE']
    print('Employment growth rate difference in counties with +1% in establishment entry and exit rates')
    print(f'Urban: {est_urban:+.3f}%')
    print(f'Rural: {est_rural:+.3f}%')

In [ ]:
show_model(2017, 2021, [3,4,5,6,7,8,9,10,11,12])

# Dashboard

In [ ]:
w_years = widgets.IntRangeSlider(min=1978, max=2021, value=(2005, 2015), description='Years', layout=widgets.Layout(width='500px'))

uic_desc = {
    1: 'metro large',
    2: 'metro small',
    3: 'micro adj metro large',
    4: 'noncore adj metro large',
    5: 'micro adj metro small',
    6: 'noncore large adj metro small',
    7: 'noncore small adj metro small',
    8: 'micro not adj metro',
    9: 'noncore large adj micro',
    10: 'noncore small adj micro',
    11: 'noncore large not adj micro',
    12: 'noncore small not adj micro'
}

nr, nc = 4, 3
w_uic = widgets.GridspecLayout(nr, nc)
i = 1
for c in range(nc):
    for r in range(nr):
        w_uic[r, c] = widgets.Checkbox(
            description=f'{i}: {uic_desc[i]}',
            value=(i not in [1, 2]),
            layout={'width': 'max-content'})
        i += 1

w_map = widgets.Output()
w_reg = widgets.Output()

w_upd = widgets.Button(description="Update")
def update_map(caller):
    selected_uics = [int(row.description.split(':')[0]) for row in w_uic.children if row.value]
    with w_map:
        w_map.clear_output()
        df['rural'] = df['uic'].isin(selected_uics).map({True: 'rural', False: 'urban'})
        plt.close()
        df.plot(column='rural', legend=True, legend_kwds={'loc': 'lower right'});
        plt.show()

    with w_reg:
        w_reg.clear_output()
        show_model(*w_years.value, selected_uics)
        
w_upd.on_click(update_map)

update_map(None)

widgets.VBox([w_upd, w_years, w_uic, w_map, w_reg])
